In [122]:
import numpy
import nltk
import pandas as pd
import re
import os
import textblob
import collections
from textblob import Word
from textblob.tokenizers import WordTokenizer
from spellchecker import SpellChecker
from bs4 import BeautifulSoup
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
import time

spell = SpellChecker()

In [123]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
def countWords(string): 
    OUT = False
    IN = True
    state = OUT 
    wc = 0
  
    # Scan all characters one by one 
    for i in range(len(string)): 
  
        # If next character is a separator,  
        # set the state as OUT 
        if (string[i] == ' ' or string[i] == '\n' or
            string[i] == '\t'): 
            state = OUT 
  
        # If next character is not a word  
        # separator and state is OUT, then  
        # set the state as IN and increment  
        # word count 
        elif state == OUT: 
            state = IN 
            wc += 1
            
    return wc

In [124]:
'''
remove encoding
bad formatting
newline character
spell check
'''

'\nremove encoding\nbad formatting\nnewline character\nspell check\n'

In [125]:
print(os.listdir('twilightzone'))

['A_Stop_at_Willoughby.txt', "It's_a_Good_Life.txt", 'Nick_of_Time.txt', 'Nightmare_At_20000_Feet.txt', 'Nothing_in_the_Dark.txt', 'One_for_the_Angels.txt', 'The_After_Hours.txt', 'The_Eye_of_the_Beholder.txt', 'The_Howling_Man.txt', 'The_Lonely.txt', 'The_Monsters_Are_Due_on_Maple_Street.txt', 'Time_Enough_at_Last.txt', 'To_Serve_Man.txt', 'Walking_Distance.txt', 'Where_is_Everybody.txt', 'Will_the_Real_Martian_Please_Stand_Up.txt']


In [188]:
script_list = []
title_list = []
with open('named_entity_tz.txt', 'r', encoding='utf-8') as myfile:
    ner = myfile.read()

names = []
for item in ner.split('\n'):
    names.append(item.split(',',1)[0])

spell.word_frequency.load_words(names)
    
for i in os.listdir('twilightzone'):
    with open('twilightzone/' + i, 'r', encoding='utf-8') as myfile:
        r = re.sub(r'(^[\s\w\-\"]*)(1)',r'\2',str(myfile.read()))
        t = textblob.TextBlob(r, tokenizer=WordTokenizer())
        
#         eh = [w for w in t.noun_phrases if countWords(w) <= 2]
#         for i in t.words:
#             if (str(i) != spell.correction(str(i))):
#                 print(i, spell.correction(str(i)))

        script_list.append(r.replace('\n',' ').lower())
        
        
        title_list.append(i.replace('_',' ').replace('.txt',''))    

In [182]:
script_list[0]

TextBlob("1. int. conference room [day]  this is a big, sumptuous, paneled room with a long, low slung table in the center surrounded by ad agency execs.  2. high angle shot looking down  at the men around the table. slow pan down until we're looking across and down the table at eye level of the men around it.  3. reverse angle looking toward the head of table  at misrell, the president of the ad agency, a fat and overpowering man of giant stature with small piggish eyes, a perpetually angry and dour face, a challenging look that never leaves. pan down the table at the faces of the men sitting there quietly and finally winding up on gart williams, a tall, attractive man in a brooks brothers suit who drums on the table nervously, checks an expensive looking watch, then rises.  4. track shot over with him  to a telephone which he picks up, seems to cup in his hand to keep the others from overhearing his conversation. he's obviously distraught beyond any kind of description now. his face 

In [183]:
wiki = 'https://en.wikipedia.org/wiki/List_of_The_Twilight_Zone_(1959_TV_series)_episodes#Concept_(1958)'
s = simple_get(wiki)
html = BeautifulSoup(s, 'html.parser')

In [184]:
table = html.find_all('table',{'class':'wikitable plainrowheaders wikiepisodetable'})
table = str(table)
markup = table.encode("utf-8")
content = markup.decode("utf-8", "ignore")
table2 = content.replace('\n','')
d = re.findall(r'(<\/td><td style="text-align:center">)([\w\s\,]*)(<span)',str(table2))
t = re.findall(r'title="([\w\s\?]*)"', str(table))

In [185]:
dates_for_all = []
for i,p in enumerate(d):
    dates_for_all.append(str(p[1]).replace('\xa0',' '))
    
eps = [29,73,6,122,80,1,33,41,40,6,21,7,88,4,0,63]
dates_list = []
for i in eps:
    dates_list.append(dates_for_all[i])

In [186]:
#  'A Stop at Willoughby',   29              May 6, 1960
#  "It's a Good Life",       73              Nov 3, 1961
#  'Nick of Time',           6              Nov 18, 1960
#  'Nightmare At 20000 Feet',122              Oct 11, 1963
#  'Nothing in the Dark',    80              Jan 5, 1962
#  'One for the Angels',     1             Oct 9, 1959
#  'The After Hours',        33              June 10, 1960
#  'The Eye of the Beholder',41              Nov 11, 1960
#  'The Howling Man',        40              Nov 4, 1960
#  'The Lonely',             6              Nov 13, 1959
#  'The Monsters Are Due on Maple Street', 21 March 4, 1960
#  'Time Enough at Last',                  Nov 20, 1959
#  'To Serve Man',                         March 2, 1962
#  'Walking Distance',                     Oct 30, 1959
#  'Where is Everybody',                   Oct 2, 1959
#  'Will the Real Martian Please Stand Up  May 26, 1961

In [192]:
data = {'Source': 'Twilight Zone', 
        'Title':title_list, 
        'Date':dates_list, 
        'Text': script_list}
tz = pd.DataFrame(data)

In [202]:
os.getcwd()

'C:\\Users\\ced4689\\Desktop\\TVF'

In [208]:
tz.to_csv(path_or_buf=,sep=('|'))

'|Source|Title|Date|Text\r\n0|Twilight Zone|A Stop at Willoughby|May 6, 1960|"1. int. conference room [day]  this is a big, sumptuous, paneled room with a long, low slung table in the center surrounded by ad agency execs.  2. high angle shot looking down  at the men around the table. slow pan down until we\'re looking across and down the table at eye level of the men around it.  3. reverse angle looking toward the head of table  at misrell, the president of the ad agency, a fat and overpowering man of giant stature with small piggish eyes, a perpetually angry and dour face, a challenging look that never leaves. pan down the table at the faces of the men sitting there quietly and finally winding up on gart williams, a tall, attractive man in a brooks brothers suit who drums on the table nervously, checks an expensive looking watch, then rises.  4. track shot over with him  to a telephone which he picks up, seems to cup in his hand to keep the others from overhearing his conversation. he